## 🎓 DSP + KNN: use simmilar Questions to add better in-context examples

This notebook illustrates the KNN-based prompt picking for Demonstrate stage of DSP framework.

There are two types of supported embedders:
1. with the help of models from `sentence-transformers` library;
2. using OpenAI's paid API. 

For the first option, you need to install `sentence-transformers` using pip (`pip install sentence-transformers`). Please check this [page](https://www.sbert.net/) to check the installation instructions. This library allows you to use a set of pre-trained feature extractors so that it will convert sentences to embeddings under the hood.

For both options, you need to install the FAISS library to perform KNN/ANN (Approximate Nearest Neighbor) search.

You can install the CPU version of the library with `pip install faiss-cpu`. This is enough for most of the use-cases, especially if your dataset of Questions and Answers is small (<1M samples).

For the official instructions (on FAISS with GPU support) and guidelines, please check [this doc page](https://github.com/facebookresearch/faiss/blob/main/INSTALL.md). 

In [1]:
import dsp
import os
import openai
# api_key = '<replace with your API key>'
# openai.api_key = api_key

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

In [2]:
train = [('Who produced the album that included a re-recording of "Lithium"?', ['Butch Vig']),
         ('Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?', ['Kevin Greutert']),
         ('The heir to the Du Pont family fortune sponsored what wrestling team?', ['Foxcatcher', 'Team Foxcatcher', 'Foxcatcher Team']),
         ('In what year was the star of To Hell and Back born?', ['1925']),
         ('Which award did the first book of Gary Zukav receive?', ['U.S. National Book Award', 'National Book Award']),
         ('What city was the victim of Joseph Druces working in?', ['Boston, Massachusetts', 'Boston']),]

train = [dsp.Example(question=question, answer=answer) for question, answer in train]

## Case 1: precomputed embeddings

If you have precomputed embeddings, you can manually assign a vector for each of the Examples. 

For the demonstration reason, we will shift some embeddings so that we will know they aren't the closest ones for some queries.

In [3]:
import numpy as np

EMB_DIM = 768
np.random.seed(42)


three_closest_embeddings = np.random.randn(3, EMB_DIM)
# shift the last three so they won't be the closest ones
three_furthest_embeddings = three_closest_embeddings.copy() + 10
train_embeddings = np.concatenate([three_closest_embeddings, three_furthest_embeddings], axis=0)

for idx, example in enumerate(train):
    example.vectorized = train_embeddings[idx]

In [4]:
dev = [('Who has a broader scope of profession: E. L. Doctorow or Julia Peterkin?', ['E. L. Doctorow', 'E.L. Doctorow', 'Doctorow'])]

dev = [dsp.Example(question=question, answer=answer) for question, answer in dev]
dev[0].vectorized = train_embeddings[1] # the same as in train, 2nd will be closest 100%

In [5]:
dsp.settings.configure(vectorizer=dsp.NaiveGetFieldVectorizer())

In [6]:
knn_func = dsp.knn(train)

In [7]:
knn_res_dev = knn_func(dev[0], 3)
[train.index(i) for i in knn_res_dev]
# first 3 train examples are the closest, the first one with index 1 is the first in the list

[1, 0, 2]

In [8]:
knn_res_train = knn_func(train[-2], 3)
[train.index(i) for i in knn_res_train]
# as expected, Eamples 0-1-2 can't be among 3 closest, because they were shifted

[4, 3, 5]

## Case 2: compute embeddings using pre-trained Sentence-Transformers

In this case, the model will be downloaded & used to extract features from the question text.

In [9]:
# we need to add some relevant and irrelevant questions to double-check the logic

train = [
    # base questions
    ('Who produced the album that included a re-recording of "Lithium"?', ['Butch Vig']),
    ('Who was the director of the 2009 movie featuring Peter Outerbridge as William Easton?', ['Kevin Greutert']),
    ('The heir to the Du Pont family fortune sponsored what wrestling team?', ['Foxcatcher', 'Team Foxcatcher', 'Foxcatcher Team']),
    ('In what year was the star of To Hell and Back born?', ['1925']),
    ('Which award did the first book of Gary Zukav receive?', ['U.S. National Book Award', 'National Book Award']),
    ('What city was the victim of Joseph Druces working in?', ['Boston, Massachusetts', 'Boston']),
    # extra, with the same entities (persons) mentioned
    ('Who founded Smart Studios in Madison, while still performing drums in Spooner at night and driving a taxi cab during the day?', ['Butch Vig']),
    ('There were two founders of recording studio called "Smart Studios". The first one was Butch Vig. Who was the second founder?', ['Steve Marker']),

    ('What is the first book of Gary Zukav?', ['"The Dancing Wu Li Masters"', "'The Dancing Wu Li Masters'", "The Dancing Wu Li Masters"]),
    ('When did Zukav returned to the United States?', ['1970'])
]

train = [dsp.Example(question=question, answer=answer) for question, answer in train]

In [10]:
QUERY_IDX = 0

# context manager is working properly
with dsp.settings.context(vectorizer=dsp.SentenceTransformersVectorizer()):
    knn_func = dsp.knn(train)

    knn_res_train_vec = knn_func(train[QUERY_IDX], 3)
    print(train[QUERY_IDX].question)  # the query itself
    # similar questions
    print('-'*80)
    for i in [train[train.index(i)].question for i in knn_res_train_vec]:
        print(i)

Who produced the album that included a re-recording of "Lithium"?
--------------------------------------------------------------------------------
Who produced the album that included a re-recording of "Lithium"?
Who founded Smart Studios in Madison, while still performing drums in Spooner at night and driving a taxi cab during the day?
There were two founders of recording studio called "Smart Studios". The first one was Butch Vig. Who was the second founder?


In [11]:
# still Naive, not SentenceTransformers - because context has ended its scope
dsp.settings.vectorizer

In [12]:
# set new vectorizer for all calls
dsp.settings.configure(vectorizer=dsp.SentenceTransformersVectorizer())

In [13]:
QUERY_IDX = 4

knn_res_train_vec = knn_func(train[QUERY_IDX], 3)
print(train[QUERY_IDX].question)  # the query itself
# similar questions
print('-'*80)
for i in [train[train.index(i)].question for i in knn_res_train_vec]:
    print(i)

Which award did the first book of Gary Zukav receive?
--------------------------------------------------------------------------------
Which award did the first book of Gary Zukav receive?
What is the first book of Gary Zukav?
When did Zukav returned to the United States?


## Case 3: compute embeddings using paid API from OpenAI

In this case, we will send Questions to the API endpoint and get embeddings to search among.

In [14]:
QUERY_IDX = 0

# let's try OpenAI embeddings
with dsp.settings.context(vectorizer=dsp.OpenAIVectorizer()):
    print(dsp.settings.vectorizer)
    knn_func = dsp.knn(train)

    knn_res_train_vec = knn_func(train[QUERY_IDX], 3)
    print(train[QUERY_IDX].question)  # the query itself
    # similar questions
    print('-'*80)
    for i in [train[train.index(i)].question for i in knn_res_train_vec]:
        print(i)

# the order is different from embeddings from SentenceTransformers

Who produced the album that included a re-recording of "Lithium"?
--------------------------------------------------------------------------------
Who produced the album that included a re-recording of "Lithium"?
There were two founders of recording studio called "Smart Studios". The first one was Butch Vig. Who was the second founder?
Who founded Smart Studios in Madison, while still performing drums in Spooner at night and driving a taxi cab during the day?
